# McKinney Chapter 10 - Data Aggregation and Group Operations

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import yfinance as yf

In [2]:
%precision 4
pd.options.display.float_format = '{:.4f}'.format
%config InlineBackend.figure_format = 'retina'

## Introduction

Chapter 10 of @mckinney2022python discusses groupby operations, the pandas equivalent of pivot tables in Excel.
Pivot tables calculate statistics (e.g., sums, means, and medians) for one set of variables by groups of other variables (e.g., weekdays and tickers).
For example, we could use a pivot table to calculate mean daily stock returns by weekday.

We will focus on:

1. The `.groupby()` method to group by columns and indexes
1. The `.agg()` method to aggregate columns to single values
1. The `.pivot_table()` method as an alternative to `.groupby()`

***Note:*** 
Indented block quotes are from @mckinney2022python unless otherwise indicated. 
The section numbers here differ from @mckinney2022python because we will only discuss some topics.

## GroupBy Mechanics

"Split-apply-combine" is an excellent way to describe pandas groupby operations.

> Hadley Wickham, an author of many popular packages for the R programming 
language, coined the term split-apply-combine for describing group operations. In the
first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide.
The splitting is performed on a particular axis of an object. For example, a DataFrame
can be grouped on its rows (axis=0) or its columns (axis=1). Once this is done, a
function is applied to each group, producing a new value. Finally, the results of all
those function applications are combined into a result object. The form of the resulting object will usually depend on what’s being done to the data. See Figure 10-1 for a
mockup of a simple group aggregation.

[Figure 10-1](https://wesmckinney.com/book/data-aggregation#fig-figure_groupby_agg) visualizes a split-apply-combine operation that:

1. Splits by the `key` column (i.e., "groups by `key`")
2. Applies the sum operation to the `data` column (i.e., "and sums `data`")
3. Combines the grouped sums (i.e., "combines the output")

We could describe this operation as "sum the `data` column by groups of the `key` column then combines the output."

In [3]:
np.random.seed(42)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.4967,-0.2341
1,a,two,-0.1383,1.5792
2,b,one,0.6477,0.7674
3,b,two,1.5230,-0.4695
4,a,one,-0.2342,0.5426


Here is the manual way to calculate the means of `data1` by groups of `key1`.

In [4]:
df.loc[df['key1'] == 'a', 'data1'].mean()

0.0414

In [5]:
df.loc[df['key1'] == 'b', 'data1'].mean()

1.0854

We can do this calculation more easily!

1. Use the `.groupby()` method to group by `key1`
2. Use the `.mean()` method to calculate the mean of `data1` within each value of `key1`

In [6]:
df['data1'].groupby(df['key1']).mean()

key1
a   0.0414
b   1.0854
Name: data1, dtype: float64

We can wrap `data1` with two sets of square brackets if we prefer our result as a data frame instead of a series.

In [7]:
df[['data1']].groupby(df['key1']).mean()

,data1
key1,
a,0.0414
b,1.0854


We can group by more than one variable!

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.1313
      two    -0.1383
b     one     0.6477
      two     1.5230
Name: data1, dtype: float64

We can use the `.unstack()` method if we want to use both rows and columns to organize data.
Recall that the `.unstack()` method un-stacks the inner index level (i.e., `level = -1`) by default so that `key2` values become the columns.

In [9]:
means.unstack()

key2,one,two
key1,,
a,0.1313,-0.1383
b,0.6477,1.5230


Our grouping variables are typically columns in the data frame we want to group, so the following syntax is more compact and easier to read.

In [10]:
df.groupby(['key1', 'key2'])['data1'].mean().unstack()

key2,one,two
key1,,
a,0.1313,-0.1383
b,0.6477,1.5230


We can wrap long chains in parentheses to insert line breaks and improve readability.

In [11]:
(
    df
    .groupby(['key1', 'key2'])
    ['data1']
    .mean()
    .unstack()
)

key2,one,two
key1,,
a,0.1313,-0.1383
b,0.6477,1.5230


*However, we must pass only numerical columns to numerical aggregation methods.*
Otherwise, pandas will give a type error.
For example, in the following code, pandas unsuccessfully tries to calculate the mean of string `key2`.

In [12]:
# df.groupby('key1').mean() # TypeError: agg function failed [how->mean,dtype->object]

We avoid this error by slicing the numerical columns.

In [13]:
df.groupby('key1')[['data1', 'data2']].mean()

,data1,data2
key1,,
a,0.0414,0.6292
b,1.0854,0.1490


### Grouping with Functions

We can also group with functions.
Below, we group with the `len` function, which calculates the lengths of the labels in the row index.

In [14]:
np.random.seed(42)
people = pd.DataFrame(
    data=np.random.randn(5, 5), 
    columns=['a', 'b', 'c', 'd', 'e'], 
    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis']
)

people

,a,b,c,d,e
Joe,0.4967,-0.1383,0.6477,1.5230,-0.2342
Steve,-0.2341,1.5792,0.7674,-0.4695,0.5426
Wes,-0.4634,-0.4657,0.2420,-1.9133,-1.7249
Jim,-0.5623,-1.0128,0.3142,-0.9080,-1.4123
Travis,1.4656,-0.2258,0.0675,-1.4247,-0.5444


In [15]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.5290,-1.6168,1.2039,-1.2983,-3.3714
5,-0.2341,1.5792,0.7674,-0.4695,0.5426
6,1.4656,-0.2258,0.0675,-1.4247,-0.5444


We can mix functions, lists, dictionaries, etc., as arguments to the `.groupby()` method.

In [16]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a       b      c       d       e
3 one -0.4634 -0.4657 0.2420 -1.9133 -1.7249
  two -0.5623 -1.0128 0.3142 -0.9080 -1.4123
5 one -0.2341  1.5792 0.7674 -0.4695  0.5426
6 two  1.4656 -0.2258 0.0675 -1.4247 -0.5444

In [17]:
d = {'Joe': 'a', 'Jim': 'b'}
people.groupby([len, d]).min()

a       b      c       d       e
3 a  0.4967 -0.1383 0.6477  1.5230 -0.2342
  b -0.5623 -1.0128 0.3142 -0.9080 -1.4123

In [18]:
d_2 = {'Joe': 'Cool', 'Jim': 'Nerd', 'Travis': 'Cool'}
people.groupby([len, d_2]).min()

a       b      c       d       e
3 Cool  0.4967 -0.1383 0.6477  1.5230 -0.2342
  Nerd -0.5623 -1.0128 0.3142 -0.9080 -1.4123
6 Cool  1.4656 -0.2258 0.0675 -1.4247 -0.5444

### Grouping by Index Levels

We can also group by index levels.

In [19]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns).transpose()
hier_df

0       1       2       3
cty tenor                                
US  1      0.1109 -0.6017 -1.2208  0.7385
    3     -1.1510  1.8523  0.2089  0.1714
    5      0.3757 -0.0135 -1.9597 -0.1156
JP  1     -0.6006 -1.0577 -1.3282 -0.3011
    3     -0.2917  0.8225  0.1969 -1.4785

In [20]:
hier_df.groupby(level='cty').count()

,0,1,2,3
cty,,,,
JP,2,2,2,2
US,3,3,3,3


In [21]:
hier_df.groupby(level='tenor').count()

,0,1,2,3
tenor,,,,
1,2,2,2,2
3,2,2,2,2
5,1,1,1,1


## Data Aggregation

***Table 10-1*** summarizes the optimized groupby methods:

- `count`: Number of non-NA values in the group
- `sum`: Sum of non-NA values
- `mean`: Mean of non-NA values
- `median`: Arithmetic median of non-NA values
- `std`, `var`: Unbiased (n – 1 denominator) standard deviation and variance
- `min`, `max`: Minimum and maximum of non-NA values
- `prod`: Product of non-NA values
- `first`, `last`: First and last non-NA values

These optimized methods are fast and efficient.
Still, pandas lets us use non-optimized methods.
First, any series method is available.

In [22]:
df

,key1,key2,data1,data2
0,a,one,0.4967,-0.2341
1,a,two,-0.1383,1.5792
2,b,one,0.6477,0.7674
3,b,two,1.5230,-0.4695
4,a,one,-0.2342,0.5426


In [23]:
df.groupby('key1')['data1'].quantile(0.9)

key1
a   0.3697
b   1.4355
Name: data1, dtype: float64

Second, we can write functions and pass them to the `.agg()` method.
These functions should accept an array and return a single value.

In [24]:
def max_minus_min(arr):
    return arr.max() - arr.min()

In [25]:
df.sort_values(by=['key1', 'data1'])

,key1,key2,data1,data2
4,a,one,-0.2342,0.5426
1,a,two,-0.1383,1.5792
0,a,one,0.4967,-0.2341
2,b,one,0.6477,0.7674
3,b,two,1.5230,-0.4695


In [26]:
df.groupby('key1')['data1'].agg(max_minus_min)

key1
a   0.7309
b   0.8753
Name: data1, dtype: float64

Some other methods work, too, even if they do not aggregate an array to a scalar.

In [27]:
df.groupby('key1')['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0000,0.0414,0.3972,-0.2342,-0.1862,-0.1383,0.1792,0.4967
b,2.0000,1.0854,0.6190,0.6477,0.8665,1.0854,1.3042,1.5230


The `.agg()` method provides two more handy features:

1. We can pass multiple functions to operate on all columns
2. We can pass specific functions to operate on specific columns

First, here are examples of multiple functions that operate on all columns.

In [28]:
df.groupby('key1')['data1'].agg(['mean', 'median', 'min', 'max'])

,mean,median,min,max
key1,,,,
a,0.0414,-0.1383,-0.2342,0.4967
b,1.0854,1.0854,0.6477,1.5230


In [29]:
df.groupby('key1')[['data1', 'data2']].agg(['mean', 'median', 'min', 'max'])

data1                         data2                      
       mean  median     min    max   mean median     min    max
key1                                                           
a    0.0414 -0.1383 -0.2342 0.4967 0.6292 0.5426 -0.2341 1.5792
b    1.0854  1.0854  0.6477 1.5230 0.1490 0.1490 -0.4695 0.7674

Second, here are examples of specific functions that operate on specific columns.

In [30]:
df.groupby('key1').agg({'data1': 'mean', 'data2': 'median'})

,data1,data2
key1,,
a,0.0414,0.5426
b,1.0854,0.1490


We must do a little more work if we want to rename `data1` and `data2` to remind ourselves which is the mean and median.

In [31]:
df.groupby('key1').agg(
    data1_mean=('data1', 'mean'),
    data2_median=('data2', 'median')
)

,data1_mean,data2_median
key1,,
a,0.0414,0.5426
b,1.0854,0.1490


We can calculate the mean *and standard deviation* of `data1` and the median of `data2` by `key1`.

In [32]:
df.groupby('key1').agg({'data1': ['mean', 'std'], 'data2': 'median'})

data1         data2
       mean    std median
key1                     
a    0.0414 0.3972 0.5426
b    1.0854 0.6190 0.1490

## Apply: General split-apply-combine

The `.agg()` method aggregates an array to a scalar.
We can use the `.apply()` method for more general calculations that do not return a scalar.
For example, the following `top()` function selects the top `n` rows in data frame `x` sorted by column `col`.
The `.sort_values()` method sorts from low to high by default.

In [33]:
def top(x, col, n=1):
    return x.sort_values(col).head(n)

In [34]:
df

,key1,key2,data1,data2
0,a,one,0.4967,-0.2341
1,a,two,-0.1383,1.5792
2,b,one,0.6477,0.7674
3,b,two,1.5230,-0.4695
4,a,one,-0.2342,0.5426


The following code returns the one row with the smallest value of `data1` within each group of `key1`.

In [35]:
df.groupby('key1').apply(top, col='data1', include_groups=False)

,,key2,data1,data2
key1,,,,
a,4,one,-0.2342,0.5426
b,2,one,0.6477,0.7674


The following code returns the *two rows* with the smallest values of `data1` within each group of `key1`.

In [36]:
df.groupby('key1').apply(top, col='data1', n=2, include_groups=False)

key2   data1   data2
key1                       
a    4  one -0.2342  0.5426
     1  two -0.1383  1.5792
b    2  one  0.6477  0.7674
     3  two  1.5230 -0.4695

We must use the `.reset_index()` method if we want to drop the index from `df`.

In [37]:
(
    df
    .groupby('key1')
    .apply(top, col='data1', n=2, include_groups=False)
    .reset_index(level=1, drop=True)
)

,key2,data1,data2
key1,,,
a,one,-0.2342,0.5426
a,two,-0.1383,1.5792
b,one,0.6477,0.7674
b,two,1.5230,-0.4695


::: {.callout-note}

The `.agg()` and `.apply()` methods both operate on groups created by the `.groupby()` method.
However, they serve different purposes and have distinct use cases.

The `.agg()` method is designed for aggregating data, meaning it applies functions that reduce a group to a single value (e.g., mean, sum, or custom functions that return a single scalar).
This method is useful for summarizing data across groups.

In contrast, the `.apply()` method is more general and flexible.
The `.apply()` method returns results of varying shapes.
The `.agg()` method is limited to scalar outputs for each group, but the `.apply()` method is not.

:::

## Pivot Tables and Cross-Tabulation

Above, we manually made pivot tables with the `.groupby()`, `.agg()`, `.apply()` and `.unstack()` methods.
pandas provides Excel-style aggregations with the `.pivot_table()` method and the `pandas.pivot_table()` function.
It is worthwhile to read the `.pivot_table()` docstring several times.

In [38]:
ind = (
    yf.download(tickers='^GSPC ^DJI ^IXIC ^FTSE ^N225 ^HSI')
    .rename_axis(columns=['Variable', 'Index'])
    .stack(future_stack=True)
)

ind.head()

[*********************100%***********************]  6 of 6 completed


Variable          Adj Close   Close    High     Low    Open  Volume
Date       Index                                                   
1927-12-30 ^DJI         NaN     NaN     NaN     NaN     NaN     NaN
           ^FTSE        NaN     NaN     NaN     NaN     NaN     NaN
           ^GSPC    17.6600 17.6600 17.6600 17.6600 17.6600  0.0000
           ^HSI         NaN     NaN     NaN     NaN     NaN     NaN
           ^IXIC        NaN     NaN     NaN     NaN     NaN     NaN

[**********************67%*******                ]  4 of 6 completed

[**********************83%***************        ]  5 of 6 completed

[*********************100%***********************]  6 of 6 completed

Variable          Adj Close   Close    High     Low    Open  Volume
Date       Index                                                   
1927-12-30 ^DJI         NaN     NaN     NaN     NaN     NaN     NaN
           ^FTSE        NaN     NaN     NaN     NaN     NaN     NaN
           ^GSPC    17.6600 17.6600 17.6600 17.6600 17.6600  0.0000
           ^HSI         NaN     NaN     NaN     NaN     NaN     NaN
           ^IXIC        NaN     NaN     NaN     NaN     NaN     NaN

The default aggregation function for `.pivot_table()` is `.mean()`.

In [39]:
ind.loc['2015':].pivot_table(index='Index')

Variable,Adj Close,Close,High,Low,Open,Volume
Index,,,,,,
^DJI,27706.3194,27706.3194,27850.4849,27546.7680,27702.7036,299024855.2101
^FTSE,7141.7548,7141.7548,7182.5771,7100.0572,7141.1015,813188499.5647
^GSPC,3358.1502,3358.1502,3375.4918,3338.4761,3357.6779,4005763661.6362
^HSI,23826.9733,23826.9733,23991.8855,23657.6522,23841.0154,2121811591.9984
^IXIC,9862.4747,9862.4747,9927.2504,9788.5201,9861.0625,3503727429.7061
^N225,24831.9359,24831.9359,24967.3558,24687.4126,24831.9677,97743271.9836


In [40]:
ind.loc['2015':].pivot_table(index='Index', aggfunc='median')

Variable,Adj Close,Close,High,Low,Open,Volume
Index,,,,,,
^DJI,26692.0947,26692.0947,26816.6045,26561.1299,26699.8799,303325000.0000
^FTSE,7256.0000,7256.0000,7289.5000,7215.7998,7255.5000,765527900.0000
^GSPC,3005.5850,3005.5850,3016.7800,2991.5000,3006.3800,3819130000.0000
^HSI,24249.4844,24249.4844,24381.0254,24078.3008,24265.7500,1932884000.0000
^IXIC,8520.2598,8520.2598,8539.0698,8457.1650,8475.0903,2905920000.0000
^N225,22823.2598,22823.2598,22922.8008,22728.0605,22849.9102,87300000.0000


We can use 
    `values` to select specific variables, 
    `pd.Grouper()` to sample different date windows, 
    and 
    `aggfunc` to select specific aggregation functions.

In [41]:
(
    ind
    .loc['2015':]
    .reset_index()
    .pivot_table(
        values='Close',
        index=pd.Grouper(key='Date', freq='YE'),
        columns='Index',
        aggfunc=['min', 'max']
    )
)

min                                                       \
Index            ^DJI     ^FTSE     ^GSPC       ^HSI      ^IXIC      ^N225   
Date                                                                         
2015-12-31 15666.4404 5874.1001 1867.6100 20556.5996  4506.4902 16795.9609   
2016-12-31 15660.1797 5537.0000 1829.0800 18319.5801  4266.8398 14952.0195   
2017-12-31 19732.4004 7099.2002 2257.8301 22134.4707  5429.0801 18335.6309   
2018-12-31 21792.1992 6584.7002 2351.1001 24585.5293  6192.9199 19155.7402   
2019-12-31 22686.2207 6692.7002 2447.8899 25064.3594  6463.5000 19561.9609   
2020-12-31 18591.9297 4993.8999 2237.3999 21696.1309  6860.6699 16552.8301   
2021-12-31 29982.6191 6407.5000 3700.6499 22744.8594 12609.1602 27013.2500   
2022-12-31 28725.5098 6826.2002 3577.0300 14687.0195 10213.2900 24717.5293   
2023-12-31 31819.1406 7256.8999 3808.1001 16201.4902 10305.2402 25716.8594   
2024-12-31 37266.6719 7446.2998 4688.6802 14961.1797 14510.2998 31458.4199   
2025-12-31 42392.2695 8223.9805 5868.5498 19623.3203 19280.7891        NaN   

                  max                                                       
Index            ^DJI     ^FTSE     ^GSPC       ^HSI      ^IXIC      ^N225  
Date                                                                        
2015-12-31 18312.3906 7104.0000 2130.8201 28442.7500  5218.8599 20868.0293  
2016-12-31 19974.6191 7142.7998 2271.7200 24099.6992  5487.4399 19494.5293  
2017-12-31 24837.5098 7687.7998 2690.1599 30003.4902  6994.7598 22939.1797  
2018-12-31 26828.3906 7877.5000 2930.7500 33154.1211  8109.6899 24270.6191  
2019-12-31 28645.2598 7686.6001 3240.0200 30157.4902  9022.3896 24066.1191  
2020-12-31 30606.4805 7674.6001 3756.0701 29056.4199 12899.4199 27568.1504  
2021-12-31 36488.6289 7420.7002 4793.0601 31084.9395 16057.4404 30670.0996  
2022-12-31 36799.6484 7672.3999 4796.5601 24965.5508 15832.7998 29332.1602  
2023-12-31 37710.1016 8014.2998 4783.3501 22688.9004 15099.1797 33753.3281  
2024-12-31 45014.0391 8445.7998 6090.2700 23099.7793 20173.8906 42224.0195  
2025-12-31 42735.2812 8260.0996 5943.1401 19760.2695 19614.7441        NaN